In [4]:
from transformers import BertTokenizer, BertModel
import torch
from fuzzywuzzy import fuzz
from sklearn.metrics import jaccard_score
import numpy as np
from collections import defaultdict
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from shapely.geometry import Point
from geopy.distance import geodesic
import pandas as pd
import string
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import geopandas as gpd
# connection
import os
from Levenshtein import distance as levenshtein_distance
from dotenv import load_dotenv
from shapely import Polygon, Point
import glob
import psycopg2
from shapely.wkt import dumps

load_dotenv("..\.env")
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adelia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Adelia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical 

In [5]:
def get_bert_embedding(name):
    inputs = tokenizer(name, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # [CLS] token embedding

def cal_cosine_similarity(name1, name2):
    embedding1 = get_bert_embedding(name1)
    embedding2 = get_bert_embedding(name2)
    return torch.nn.functional.cosine_similarity(embedding1, embedding2).item()

def fuzzy_similarity(name1, name2):
    return fuzz.ratio(name1.lower(), name2.lower()) / 100.0

def jaccard_similarity(name1, name2):
    set1 = set(name1.lower().split())
    set2 = set(name2.lower().split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

def combined_similarity(name1, name2, weights=None):
    if weights is None:
        weights = [0.2, 0.45, 0.55]  # Adjust these weights as needed
    fuzzy_sim = fuzzy_similarity(name1, name2)
    jac_sim = jaccard_similarity(name1, name2)

    combined_score = (
        weights[1] * fuzzy_sim +
        weights[2] * jac_sim
    )
    return combined_score

def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

def remove_extra_whitespaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def remove_stopwords(text):
    stop_words = set(stopwords.words('indonesian'))
    return ' '.join([word for word in text.split() if word not in stop_words])

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

def preprocess_text(text):
    text = to_lowercase(text)
    text = remove_special_characters(text)
    text = remove_extra_whitespaces(text)
    text = lemmatize_text(text)
    return text

def to_lowercase(text):
    return text.lower()

def split_at_indices(s, index = 3):
    return str(s[:index]) + '.' + str(s[index:])

def calculate_distance(row, gdf):
    point1 = (row.geometry.y, row.geometry.x)  # Original POI
    point2 = (gdf.loc[gdf['id'] == row['id_buffer'], 'geometry'].iloc[0].y,
              gdf.loc[gdf['id'] == row['id_buffer'], 'geometry'].iloc[0].x)  # Matched POI
    return geodesic(point1, point2).meters

def find_cluster(id, visited, cluster, group_mapping):
    visited.add(id)
    cluster.add(id)
    for neighbor in group_mapping[id]:
        if neighbor not in visited:
            find_cluster(neighbor, visited, cluster, group_mapping)

def tfidf_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

def levenshtein_similarity(text1, text2):
    max_len = max(len(text1), len(text2))
    return 1 - levenshtein_distance(text1, text2) / max_len

def overlap_coefficient(text1, text2):
    set1, set2 = set(text1.split()), set(text2.split())
    return len(set1 & set2) / min(len(set1), len(set2))

def dice_coefficient(text1, text2):
    set1, set2 = set(text1.split()), set(text2.split())
    return 2 * len(set1 & set2) / (len(set1) + len(set2))

def inputData(match):
    data3=','.join("'{0}'".format(x) for x in match)
    return data3

def capitalize_without_punctuation(text):
    punctuation_chars = set(string.punctuation)
    words = text.split()
    processed_words = []
    for word in words:
        if any(char in punctuation_chars for char in word):
            processed_words.append(word)
        else:
            processed_words.append(word.capitalize())
    return ' '.join(processed_words)


def clean_df_input(df, poi_name_col, longitude_col, latitude_col):
    """
    poi name, longitude, latitude
    """
    df = df[[poi_name_col, longitude_col, latitude_col,'id']].drop_duplicates(subset = [poi_name_col, longitude_col, latitude_col])
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[longitude_col], 
                                                           df[latitude_col], 
                                                           crs = 'epsg:4326'))
    buffer_distance_m = 30  # Distance in meters
    buffer_user = gdf.copy()
    buffer_user['geometry'] = buffer_user.buffer(buffer_distance_m / 109527.56)  # Convert meters to degrees
    if gdf.shape[0]<=1:
        return gdf, buffer_user
    else:
        buffered_gdf = gpd.sjoin(gdf, buffer_user[[poi_name_col, 'id', 'geometry']], how='inner', predicate='intersects')
        buffered_gdf = buffered_gdf.rename(columns={
            f'{poi_name_col}_left': poi_name_col, 
            f'{poi_name_col}_right': 'name_buffer',
            'id_left': 'id',
            'id_right': 'id_buffer'
        })

        buffered_gdf = buffered_gdf[~(buffered_gdf['id']==buffered_gdf['id_buffer'])]
        buffered_gdf['distance'] = buffered_gdf.apply(lambda row: calculate_distance(row, gdf), axis=1)

        similarity_threshold = 10  # Distance threshold in meters
        buffered_gdf['is_similar'] = (
            (buffered_gdf['distance'] <= similarity_threshold) &  # Distance condition
            (buffered_gdf[poi_name_col].str.lower() == buffered_gdf['name_buffer'].str.lower())  # Name match condition
        )

        similar_pois = buffered_gdf[buffered_gdf['is_similar']]
        similar_pois['pair'] = similar_pois.apply(
            lambda row: tuple(sorted([row['id'], row['id_buffer']])), axis=1
        )

        group_mapping = defaultdict(set)
        for _, row in similar_pois.iterrows():
            group_mapping[row['id']].add(row['id'])
            group_mapping[row['id']].add(row['id_buffer'])
            group_mapping[row['id_buffer']].add(row['id'])

        visited = set()
        clusters = []
        for id in group_mapping:
            if id not in visited:
                cluster = set()
                find_cluster(id, visited, cluster, group_mapping)
                clusters.append(cluster)

        representative_ids = [min(cluster) for cluster in clusters]
        representative_ids = representative_ids+gdf[~
                                                    gdf['id'].isin(similar_pois['id'].unique().tolist())
                                                   ]['id'].unique().tolist()

        cleaned_gdf = gdf[gdf['id'].isin(representative_ids)].copy()
        cleaned_gdf.reset_index(drop=True, inplace=True)
        buffer_user = buffer_user[buffer_user['id'].isin(cleaned_gdf['id'].unique().tolist())]
        return cleaned_gdf, buffer_user

def query_poi_from_buffer(buffer_gdf, conn, industry_name = None):
    # Ensure buffer_gdf is in WGS84 (EPSG:4326)
    buffer_gdf = buffer_gdf.to_crs(epsg=4326)
    
    # Create a list of WKT strings for buffer geometries
    buffer_wkt_list = buffer_gdf["geometry"].apply(dumps).tolist()
    
    # Construct the SQL query with multiple polygons
    polygon_conditions = " OR ".join([
        f"ST_Intersects(a.geom, ST_GeomFromText('{polygon}', 4326))"
        for polygon in buffer_wkt_list
    ])
    
    
    if industry_name !=None:
        industry_name = inputData(industry_name)
        filt = f"""AND e.industry_name in ({industry_name})"""
    else :
        filt = ""
    sql = f"""
    SELECT
        a.poi_id,
        a.poi_name,
        a.geom,
        e.industry_name
    FROM v3_tbl_poi a
    JOIN v3_tbl_brand b on a.brand_id = b.brand_id
    JOIN v3_tbl_category c on b.category_id = c.category_id
    JOIN v3_tbl_group d on c.group_id = d.group_id
    JOIN v3_tbl_industry e on d.industry_id = e.industry_id
    WHERE 
        {polygon_conditions} and a.status = 'T' and b.status = 'T' 
        and c.status = 'T' and d.status = 'T' and e.status = 'T' {filt};
    """
        
    return gpd.read_postgis(sql, conn, crs = 'epsg:4326')

def clean_longest_sentence(text):
    parts = text.split('|')
    cleaned_parts = [re.sub(r'\s*\(.*?\)\s*', '', part).strip() for part in parts]
    longest_sentence = max(cleaned_parts, key=len)
    return longest_sentence

def predict_poi(poi_name):
    best_model = joblib.load(r"poi_best_multioutput_model.pkl")
    vectorizer = joblib.load(r"poi_tfidf_vectorizer.pkl")
    poi_name_tfidf = vectorizer.transform([poi_name])
    predictions = best_model.predict(poi_name_tfidf)
    return {
        'brand_name': predictions[0][0],
        'category_name': predictions[0][1],
        'group_name': predictions[0][2],
        'industry_name':predictions[0][3],
    }

In [6]:
#connect to production
HOST = os.getenv('host_production')
DB = os.getenv('DB_NAME')
PORT = 5432
USER= os.getenv('username_production')
PWD = os.getenv('password_production')
conn0 = psycopg2.connect(host=HOST,database=DB, user=USER, password=PWD)

DB_NAME_V3_POI=os.getenv("DB_NAME_V3_POI")
DB_USERNAME_V3_POI=os.getenv("DB_USERNAME_V3_POI")
DB_PASSWORD_V3_POI=os.getenv("DB_PASSWORD_V3_POI")
DB_HOST_V3=os.getenv("DB_HOST_V3")
conn_poi = psycopg2.connect(host=DB_HOST_V3,
                            database=DB_NAME_V3_POI, 
                            user=DB_USERNAME_V3_POI, 
                            password=DB_PASSWORD_V3_POI)



In [7]:
df = pd.read_excel(r"data_raw.xlsx")
df['id'] = range(0, len(df))
df_raw = df.copy()

In [8]:
gdf, buffer_gdf = clean_df_input(df=df, poi_name_col='nama_merchant', longitude_col= 'longitude',latitude_col='latitude')

C:\Users\Adelia\AppData\Local\Temp\ipykernel_4024\1567011507.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffer_user['geometry'] = buffer_user.buffer(buffer_distance_m / 109527.56)  # Convert meters to degrees


In [9]:
data = gdf.copy()
buffer = buffer_gdf[buffer_gdf['id'].isin(data['id'].unique().tolist())].copy()

In [10]:
%%time
poi_internal = gpd.read_parquet("data_internal.parquet")

CPU times: total: 0 ns
Wall time: 222 ms


In [26]:
poi_join = gpd.sjoin(
    poi_internal,
    buffer[['geometry','nama_merchant','id']]
         ).drop(columns = 'index_right')
# poi_not_similar = data[~data['id'].isin(poi_join['id'].unique().tolist())]

In [27]:
x = poi_join.rename(columns = {'geom':'geometry_internal'})
x = pd.merge(x, data[['id','geometry']].rename(columns = {'geometry':'geometry_input'}), on = 'id')

# Convert to a CRS that uses meters (e.g., EPSG:3395)
x['geometry_internal'] = x['geometry_internal'].to_crs(epsg=3395)
x['geometry_input'] = x['geometry_input'].to_crs(epsg=3395)

# Calculate the distance in meters
x['distance_meters'] = x['geometry_internal'].distance(x['geometry_input'])


In [28]:
%%time
# Clean the 'poi_name' and 'nama_merchant' columns using the preprocess_text function
x['cleaned_poi_name'] = x['poi_name'].apply(preprocess_text)
x['cleaned_nama_merchant'] = x['nama_merchant'].apply(preprocess_text)

CPU times: total: 93.8 ms
Wall time: 1.29 s


In [29]:
%%time
# Calculate the similarity
df = x.copy()
df['similarity'] = df.apply(lambda row: combined_similarity(row['cleaned_poi_name'], row['cleaned_nama_merchant']), axis=1)
df['fuzzy'] = df.apply(lambda row: fuzzy_similarity(row['cleaned_poi_name'], row['cleaned_nama_merchant']), axis=1)
df['jaccard'] = df.apply(lambda row:jaccard_similarity(row['cleaned_poi_name'], row['cleaned_nama_merchant']), axis=1)
df['tfidf_similarity'] = df.apply(lambda row:tfidf_similarity(row['cleaned_poi_name'], row['cleaned_nama_merchant']), axis=1)
df['levenshtein_similarity'] = df.apply(lambda row:levenshtein_similarity(row['cleaned_poi_name'], row['cleaned_nama_merchant']), axis=1)

CPU times: total: 328 ms
Wall time: 2.13 s


In [30]:
x_columns = ['distance_meters','fuzzy','jaccard','tfidf_similarity','levenshtein_similarity']
df_predict = df[x_columns+['poi_id','id']]
X_test = df_predict[x_columns]

In [11]:
import pickle
pickled_model = pickle.load(open('model_v1.pkl', 'rb'))

In [32]:
predict = pickled_model.predict(X_test)
df_predict['target_model'] = predict
df['target_model'] = predict

C:\Users\Adelia\AppData\Local\Temp\ipykernel_21024\2077780276.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict['target_model'] = predict


In [33]:
similar = df[df['target_model']==1]
similar['similarity'] = similar[['fuzzy','jaccard','tfidf_similarity','levenshtein_similarity']].mean(axis = 1)

C:\Users\Adelia\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [34]:
first = similar[(similar['fuzzy']>0.775)&(similar['tfidf_similarity']>0.5)&
        (similar['jaccard']>0.5)&(similar['levenshtein_similarity']>0.667)]
not_ = similar[~(similar['id'].isin(first['id'].tolist()))]
second = not_[not_['similarity']>0.64]

not_2 = not_[~(not_['id'].isin(second['id'].tolist()))]
third = not_2[not_2['fuzzy']>0.9]

not_3 = not_2[~(not_2['id'].isin(third['id'].tolist()))]
fourth = not_3[(not_3['similarity']>0.6)&(not_3['tfidf_similarity']>0.6)]

not_4 = not_3[~not_3['id'].isin(fourth['id'].tolist())]

In [35]:
similar = pd.concat([first, second, third, fourth]).drop_duplicates()

In [36]:
gdf_sorted = similar.sort_values(by=['id', 'similarity', 'distance_meters'], ascending=[True, False, True])

# Drop duplicates, keeping the first occurrence (the one with max similarity and min distance)
gdf_unique = gdf_sorted.drop_duplicates(subset='id', keep='first')
poi_similar = gdf_unique[['poi_id','poi_name','nama_merchant',
                          'id','distance_meters','similarity',
                          'fuzzy','jaccard','geometry_internal']]

poi_similar['nama_merchant'] = poi_similar['nama_merchant'].apply(capitalize_without_punctuation)

C:\Users\Adelia\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [37]:
final = df_raw[['nama_merchant',
 'latitude',
 'longitude','id']]
final = final[~(final['id'].isin(poi_similar['id'].tolist()))]
final['poi_name'] = final['nama_merchant'].apply(capitalize_without_punctuation)
final['poi_name'] = final['poi_name'].apply(clean_longest_sentence)

In [ ]:
df_expanded = final['poi_name'].apply(predict_poi).apply(pd.Series)
poi_non_similar = pd.concat([final, df_expanded], axis=1)

In [ ]:
poi_non_similar.head(10)

In [ ]:
poi_similar.head(5)